# Exemple d'utilisation de la fonction update_conductivities()

### 1. Charger la fonction et importer les fichier conductivities.csv de base fourni dans le dossier : MARSHAL/inputs/
data est le dataframe contenant les conductivités de base (cad celle par défaut dans MARSHAL)

In [12]:
import pandas as pd

def update_conductivities(df, values:list, on="taproots", output_fname=None):
    ''' df = dataframe contenant les conductivités. Doit être chargé préalablement.
        values = liste de 2 éléments contenant les différentes valeurs de Kr obtenue via MECHA. L'ordre de ces valeurs est important!
                 La première valeur correspond au Kr de maturité #0 et la seconde valeur au Kr de maturité #1. Si on == allroots
                 alors values est une liste de 4 valeurs dans l'ordre suivant : 
                 [Kr mat.#0 tap root, Kr mat.#1 tap root, Kr mat.#0 lat. root, Kr mat.#1 alt root]
        on = string permettant de préciser quelles conductivités doivent être modifiées. Trois cas possibles : taproots, lateralroots
             ou allroots.
        output_fname = Nom du fichier .csv produit. Facultatif.       
    '''
    
    df_copy = df.copy(deep=True)
    taproots = ['Taproot', 'Basalroot', 'Shootborneroot']
    lateralroots = ['Lateral', 'LongLateral']
    
    if on == "taproots":
        subset = df_copy.loc[(df_copy['order'].isin(taproots)) & (df_copy['type'] == 'kr')]
        subset.loc[(subset['x'] == 0.0) | (subset['x'] == 9.0), 'y'] = values[0]    #set value to Kr matutity level #0
        subset.loc[(subset['x'] == 20.0) | (subset['x'] == 50.0), 'y'] = values[1]  #set value to Kr matutity level #1
        #replace old values by the new ones
        idx = subset.index
        df_copy.loc[idx, 'y'] = subset['y']
        
    elif on == "lateralroots":
        subset = df_copy.loc[(df_copy['order'].isin(lateralroots)) & (df_copy['type'] == 'kr')]
        subset.loc[(subset['x'] == 0.0) | (subset['x'] == 13.0), 'y'] = values[0]   #set value to Kr matutity level #0
        subset.loc[(subset['x'] == 13.1) | (subset['x'] == 50.0), 'y'] = values[1]  #set value to Kr matutity level #1
        #replace old values by the new ones
        idx = subset.index
        df_copy.loc[idx, 'y'] = subset['y']
        
    elif on == "allroots":
        subset_tap = df_copy.loc[(df_copy['order'].isin(taproots)) & (df_copy['type'] == 'kr')]
        subset_tap.loc[(subset_tap['x'] == 0.0) | (subset_tap['x'] == 9.0), 'y'] = values[0]    #set value to Kr matutity level #0
        subset_tap.loc[(subset_tap['x'] == 20.0) | (subset_tap['x'] == 50.0), 'y'] = values[1]  #set value to Kr matutity level #1
        subset_lat = df_copy.loc[(df_copy['order'].isin(lateralroots)) & (df_copy['type'] == 'kr')]
        subset_lat.loc[(subset_lat['x'] == 0.0) | (subset_lat['x'] == 13.0), 'y'] = values[2]   #set value to Kr matutity level #0
        subset_lat.loc[(subset_lat['x'] == 13.1) | (subset_lat['x'] == 50.0), 'y'] = values[3]  #set value to Kr matutity level #1
        #replace old values by the new ones
        idx_tap = subset_tap.index
        idx_lat = subset_lat.index
        df_copy.loc[idx_tap, 'y'] = subset_tap['y']
        df_copy.loc[idx_lat, 'y'] = subset_lat['y']
        

    #write new csv file
    if output_fname != None:
        df_copy.to_csv(str(output_fname)+'.csv', sep=',', index=False)
    else:
        df_copy.to_csv(str(modified_conductivities)+'.csv', sep=',', index=False)

# path à remplacer 
data = pd.read_csv('/Users/mattias/Documents/cours/master/Q2/Modelisation des systemes biologiques/devoir_S6/modelisation-anatomie-architecture-mattiasve/MARSHAL/inputs/conductivities.csv', 
                   sep=',')
data.head(12)

,id,order_id,order,type,x,y
0,1,1,Taproot,kr,0.0,0.000181
1,2,1,Taproot,kr,9.0,0.000181
2,3,1,Taproot,kr,20.0,0.000017
3,4,1,Taproot,kr,50.0,0.000017
4,5,1,Taproot,kx,0.0,0.029500
5,6,1,Taproot,kx,9.0,0.029500
6,7,1,Taproot,kx,20.0,4.320000
7,8,1,Taproot,kx,50.0,4.320000
8,9,2,Lateral,kr,0.0,0.000181
9,10,2,Lateral,kr,13.0,0.000181


### 2. Créer le csv avec les données de base correspondant à une anatomie spécifique

Une fois l'anatomie des racines principales et latérales construite via GRANAR et que les différentes valeurs respectives de $K_r$ sont calculées via MECHA, on peut les stocker dans la liste `base_kr`, en respectant l'ordre spécifié dans la documentation. Le csv contenant les conductivités de base correspondant à une anatomie spécifique est alors construit en utilisant la liste `base_kr` et l'argument `on=allroots`. 

Le ficher base_conductivities.csv est alors le fichier de base sur lequel les différentes valeurs de $K_r$ des racines latérales **ou** principales seront modifiées

In [10]:
base_kr = [0.000114888315352683, 0.0000380133951336158, 0.000125871860736347, 0.0000379739887261723]
update_conductivities(data, base_kr, on="allroots", output_fname='base_conductivities')

### 3. Charger le csv de base et modification

On charge maintenat le csv de base afin de pouvoir modifier les valeurs de $K_r$ après avoir modifié les paramètres anatomiques. Si les modifications sont faites sur les racines principales alors `on=taproots`, sinon `on=latroots`. Ne pas oublier de renommer à chaque fois le nouveau csv afin de ne pas tout mélanger!

Le nouveau csv peut alors être directement mis dans MARSHAL et cette étape peut être refaite autant de fois que nécessaire! :)

In [11]:
#importer csv de base
base_cond = pd.read_csv('/Users/mattias/Documents/cours/master/Q2/Modelisation des systemes biologiques/devoir_S6/modelisation-anatomie-architecture-mattiasve/update_conductivities/base_conductivities.csv',
                        sep=',')
#nouvelles valeurs de Kr obtenue via MECHA
kr_tap_25 = [0.00010199593517329163, 3.453232081480753e-05]
update_conductivities(base_cond, kr_tap_25, on="latroots", output_fname='conductivities_tap25')
